In [1]:
from keras import applications, optimizers
from keras.layers import GlobalAveragePooling2D, Dense
from keras.models import Sequential
from keras.datasets import mnist
from keras.layers.convolutional import Convolution2D
from keras import backend as K
import keras
import numpy as np
from keras.applications.imagenet_utils import preprocess_input
from img_loader import DatasetLoader
from keras.utils import np_utils
from keras.preprocessing import image
from keras.models import Model
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image, ImageOps
from sklearn.preprocessing import MinMaxScaler
import scipy.misc
from PIL import ImageEnhance
import cv2
import matplotlib.pyplot as pp
from keras.models import load_model, save_model
import random
from numpy import argmax

Using TensorFlow backend.


In [2]:
%matplotlib inline
np.random.seed(123)  # for reproducibility
random.seed(123)

In [3]:
dl = DatasetLoader('/home/lux/dev/Bachelor-2017/keras/mnist_png', 10000)

DATASET LOADER] Discovering dataset...
DATASET LOADER] 
10 classes found.
 68992 images found.
DATASET LOADER] Shuffling order...
DATASET LOADER] 
Ready for loading!
 51744 for training and 17248 for testing


In [4]:
i = 0
img = cv2.imread(dl.baseDirectory + "/" + dl.imgDataArray[i].directory + "/" +
                                       dl.imgDataArray[i].name, cv2.IMREAD_COLOR)
i = 1
img2 = cv2.imread(dl.baseDirectory + "/" + dl.imgDataArray[i].directory + "/" +
                                       dl.imgDataArray[i].name, cv2.IMREAD_COLOR)
img = img.astype('float32') # Okay don't worry it's juste the web viewer that messes up with the colors nothing else.
cv2.imwrite('out.png', img)

True

In [5]:
img2 = img * 0.5
cv2.imwrite('out2.png', img2)

True

In [6]:
img3 = img + img2
cv2.imwrite('out3.png', img3)

True

In [7]:
model = load_model("/home/lux/dev/Bachelor-2017/keras/mnist.h5")

/usr/lib64/python3.5/site-packages/keras/models.py:281: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [24]:
model.get_layer("W").get_weights()

[array([[ -1.69540837e-01,   1.01465225e-01,   1.26929581e-01,
           1.17718138e-01,  -1.37613907e-01,   2.41278023e-01,
          -2.78581411e-01,   2.49608979e-01,   1.82236806e-01,
          -2.33479798e-01],
        [  1.53826535e-01,  -1.63081154e-01,  -4.15028743e-02,
           2.57200673e-02,  -1.02355346e-01,   2.79588640e-01,
          -1.10236041e-01,  -2.72446033e-02,   8.10337588e-02,
           8.48360732e-02],
        [ -1.37318835e-01,  -1.65765453e-02,   1.94716044e-02,
           9.91193727e-02,   1.05551757e-01,   4.51466218e-02,
          -7.32123479e-02,  -7.12150857e-02,   5.84463924e-02,
           1.19901307e-01],
        [ -1.59582943e-01,   6.63695559e-02,  -1.22805536e-01,
          -2.79621691e-01,   1.23497002e-01,  -7.02556819e-02,
           1.69604838e-01,   8.06085840e-02,   1.03356510e-01,
          -2.26449743e-01],
        [  1.42019510e-01,   1.27802655e-01,   1.92950368e-01,
          -6.27806364e-03,   1.00225449e-01,  -8.54364783e-02,
      

In [16]:
def get_outputs_generator(model, layer_name):
    """
    Gets the output generator of a specific layer of the model.

    :param model: The model
    :param layer_name: The layer's name
    :return: the output generator (a function)
    """
    layer_model = Model(
        input=model.input,
        output=model.get_layer(layer_name).output
    )

    return layer_model.predict

def heatmap_generate(input_img, model, class_to_predict, layer_name, image_name=None):
   
    output_generator = get_outputs_generator(model, layer_name)
    layer_outputs = output_generator(np.expand_dims(input_img, axis=0))[0]
    
    heatmap = cv2.resize(layer_outputs[:, :, 0], (224, 224))
    
    w = model.get_layer("W").get_weights()[0]
    heatmap *= w[0][class_to_predict]

    for z in range(1, layer_outputs.shape[2]): # Iterate through the number of kernels
        img = layer_outputs[:, :, z]

        deprocessed = cv2.resize(img, (224, 224))
        heatmap += deprocessed * w[z][class_to_predict]
    heatmap = MinMaxScaler((0.0, 255.0)).fit_transform(heatmap)

    heatmap_colored = cv2.applyColorMap(np.uint8(heatmap), cv2.COLORMAP_JET)

    if image_name is not None:
        heatmap_colored = cv2.putText(heatmap_colored, image_name, (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 0),
                                      2)
    return heatmap_colored

In [17]:
predict_input = np.expand_dims(img, axis=0)
predict_input = predict_input.astype('float32')
predict_input = preprocess_input(predict_input)
predict_input.shape[1:3]

(28, 28)

In [18]:
argmax(model.predict(predict_input))

4

In [19]:
heatmap = heatmap_generate(predict_input[0], model, argmax(model.predict(predict_input)), 'CAM')
heatmap
cv2.imwrite('CAM_nponly.png',heatmap )

/usr/lib/python3.5/site-packages/ipykernel/__main__.py:11: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("CA..., inputs=Tensor("co...)`


True